In [1]:
import numpy as np
import cv2
import heapq

In [2]:
h = 250
w = 600
map = np.zeros((h,w,1), dtype=np.uint8)

In [3]:
v = np.zeros((h,w,12))
v[250-1][600-1][12-1]

np.float64(0.0)

In [4]:
text_set = set()
buffer_set = set()
border_set = set()


In [5]:
radius = 5
clearance = 5
buffer = radius+clearance

In [6]:
map_text = map.copy()
map_buffer = map.copy()
map_border = map.copy()

map_text = cv2.putText(map_text, "E N P M 6 6 1",(50,150), 0, 2, 255,5)
text_coord = np.where(map_text==255)
for i in range(0,len(text_coord[0])):
    xy = (text_coord[1][i],text_coord[0][i])
    text_set.add(xy)

for item in text_set:
    map_buffer = cv2.circle(map_buffer,item,buffer,255,-1)
buffer_coord = np.where(map_buffer==255)
for i in range(0,len(buffer_coord[0])):
    xy = (buffer_coord[1][i],buffer_coord[0][i])
    if xy not in text_set:
        buffer_set.add(xy)

map_border = cv2.rectangle(map_border,(0,0),(w-1,h-2),255,1)
border_coord = np.where(map_border==255)
for i in range(0,len(border_coord[0])):
    xy = (border_coord[1][i],border_coord[0][i])
    border_set.add(xy)

del map_text
del map_buffer
del map_border
del text_coord
del buffer_coord
del border_coord








In [7]:
display_map = np.zeros((h,w,3),dtype=np.uint8)
for item in text_set:
    display_map[item[1],item[0]] = (255,0,0)

for item in buffer_set:
    display_map[item[1],item[0]] = (0,255,0)

for item in border_set:
    display_map[item[1],item[0]] = (0,0,255)



In [8]:
# open list contains (total cost, (node_x,node_y, node_t)
# tc_matrix contains total cost
# c2c_matrix contains c2c
# px_matrix contains parent x
# py_matrix contains parent y
# pt_matrix contains parent t
tc_matrix = np.zeros((h,w,12))
c2c_matrix = np.zeros((h,w,12))
px_matrix = np.zeros((h,w,12))
py_matrix = np.zeros((h,w,12))
pt_matrix = np.zeros((h,w,12))

open_list = []
closed_list = []
heapq.heapify(open_list)
heapq.heapify(closed_list)

start_x = 5
start_y = 5
start_t = 0
start_xy = (start_x,start_y)
start_xyt = (start_x,start_y,start_t)
end_x = 500
end_y = 240
end_t = 0
end_xy = (end_x,end_y)
end_xyt = (end_x,end_y,end_t)
step_size = 3




In [9]:

def distance(p1,p2):
    p1x,p1y,p1t = p1
    p2x,p2y,p1t = p2
    dist = np.sqrt(((p1x-p2x)**2)+((p1y-p2y)**2))
    #dist = int(round(dist))
    return dist

def line_cross(p1,p2):
    p1x,p1y,p1t = p1
    p2x,p2y,p2t = p2
    p1xy = (p1x,p1y)
    p2xy = (p2x,p2y)
    l_map = np.zeros((h,w,1),dtype=np.uint8)
    l_map = cv2.line(l_map,p1xy,p2xy,(255),1)
    l_coords = np.where(l_map==255)
    for i in range(0,len(l_coords[0])):
        xy = (l_coords[1][i],l_coords[0][i])
        if xy in text_set:
            return True
        elif xy in buffer_set:
            return True
        elif xy in border_set:
            return True


In [10]:
# open list contains (total cost, (node_x,node_y, node_t)
# tc_matrix contains total cost
# c2c_matrix contains c2c
# px_matrix contains parent x
# py_matrix contains parent y
# pt_matrix contains parent t
def move(node,angle):
    
    p_xyt = node[1]
    p_x,p_y,p_t = p_xyt
    p_t_index = int(p_t/30)

    c_x = int(round(p_x+(step_size*np.cos(np.deg2rad(angle)))))
    c_y = int(round(p_y+(step_size*np.sin(np.deg2rad(angle)))))
    c_t = angle
    c_t_index = int(c_t/30)
    c_xyt = (c_x,c_y,c_t)

    if line_cross(p_xyt,c_xyt):
        return
    
    current_tc= tc_matrix[c_y][c_x][c_t_index]

    p_c2c = c2c_matrix[p_y][p_x][p_t_index]
    c_c2c = p_c2c+step_size
    c_c2g = distance(c_xyt,end_xyt)
    c_tc = c_c2c+c_c2g

    if current_tc ==  0:
        tc_matrix[c_y][c_x][c_t_index] = c_tc
        c2c_matrix[c_y][c_x][c_t_index] = c_c2c
        px_matrix[c_y][c_x][c_t_index] = p_x
        py_matrix[c_y][c_x][c_t_index] = p_y
        pt_matrix[c_y][c_x][c_t_index] = p_t
        new_node = (c_tc, c_xyt)
        heapq.heappush(open_list,new_node)
    
    elif current_tc > c_tc:
        old_node = (current_tc, c_xyt)
        open_list.remove(old_node)
        tc_matrix[c_y][c_x][c_t_index] = c_tc
        c2c_matrix[c_y][c_x][c_t_index] = c_c2c
        px_matrix[c_y][c_x][c_t_index] = p_x
        py_matrix[c_y][c_x][c_t_index] = p_y
        pt_matrix[c_y][c_x][c_t_index] = p_t
        new_node = (c_tc, c_xyt)
        heapq.heappush(open_list,new_node)
        
    else:
        return




In [11]:
start_tc = distance(start_xyt,end_xyt)
first_ol = (start_tc,start_xyt)
heapq.heappush(open_list,first_ol)
tc_matrix[start_y][start_x][int(start_t/30)] = start_tc
c2c_matrix[start_y][start_x][int(start_t/30)] = 0
px_matrix[start_y][start_x][int(start_t/30)] = start_x
py_matrix[start_y][start_x][int(start_t/30)] = start_y
pt_matrix[start_y][start_x][int(start_t/30)] = start_t



In [12]:
map_display = display_map.copy()
#i = 0
while True:
    parent_node = heapq.heappop(open_list)

    p_xyt = parent_node[1]
    dist_to_goal = distance(p_xyt,end_xyt)

    if dist_to_goal < 1.5:
        break
    
    heapq.heappush(closed_list,parent_node)
    #if i%10 == 0:
    p1x,p1y,p1t = parent_node[1]
    p2x = px_matrix[p1y][p1x][int(p1t/30)]
    p2y = py_matrix[p1y][p1x][int(p1t/30)]
    p1 = (p1x,p1y)
    p2 = (int(p2x),int(p2y))
    map_display = cv2.line(map_display,p1,p2,(255,255,255),1)
    frame = map_display.copy()
    cv2.imshow("animation",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    for item in [-2,-1,0,1,2]:
        ang = p_xyt[2] + (30*item)
        ang = ang%360
        move(parent_node,ang)
    #i += 1
    
cv2.waitKey(0)
cv2.destroyAllWindows()